损失函数
===
机器学习中的所有算法都依赖于最小化或最大化某一个函数，我们称之为“目标函数”。最小化的这组函数被称为“损失函数”。损失函数是衡量预测模型预测期望结果表现的指标。寻找函数最小值的最常用方法是“梯度下降”。把损失函数想象成起伏的山脉，梯度下降就像从山顶滑下，目的是到达山脉的最低点。没有一个损失函数可以适用于所有类型的数据。损失函数的选择取决于许多因素，包括是否有离群点，机器学习算法的选择，运行梯度下降的时间效率，是否易于找到函数的导数，以及预测结果的置信度。

损失函数可以大致分为两类：分类损失(Classification Loss)和回归损失(Regression Loss)。

# 1.回归损失

## 1.1.均方误差-Mean Square Error
也可以叫做二次损失(Quadratic Loss)，或者L2损失(L2 Loss)。MSE是目标变量与预测值之间的距离平方和
$$MSE=\frac{\sum_{i=1}^n(y_i-y_i^p)^2}{n}$$
![images](Images/02/13_01_001.jpg)

In [2]:
import numpy as np

def mse(true, pred):
    return np.sum((true - pred) ** 2)

## 1.2.平均绝对误差-Mean Absolute Error
也可以叫做L1损失(L1 Loss).MAE是目标变量和预测变量之间差异绝对值之和。因此，它在一组预测中衡量误差的平均大小，而不考虑误差的方向。(如果我们也考虑方向，那将被称为平均偏差(Mean Bias Error, MBE)，它是残差或误差之和)
$$MAE=\frac{\sum_{i=1}^n\left| y_i-y_i^p \right|}{n}$$
![images](Images/02/13_01_002.jpg)

简而言之， 使用平方误差更容易求解，但使用绝对误差对离群点更加鲁棒。每当我们训练机器学习模型时，我们的目标就是找到最小化损失函数的点。当然，当预测值正好等于真实值时，这两个损失函数都达到最小值。

In [3]:
import numpy as np

def mae(true, pred):
    return np.sum(np.abs(true - pred))

## 1.3.MAE与MSE的比较

由于MSE对误差$\epsilon$进行平方操作($y - y_predicted = \epsilon$)，如果$\epsilon$ > 1，误差的值会增加很多。如果我们的数据中有一个离群点，$\epsilon$的值将会很高，将会远远大于$\left| \epsilon \right|$。这将使得和以MAE为损失的模型相比，以MSE为损失的模型会赋予更高的权重给离群点。以MSE为损失的模型将被调整以最小化这个离群数据点，但是却是以牺牲其他正常数据点的预测效果为代价，这最终会降低模型的整体性能。

对所有的观测数据，如果我们只给一个预测结果来最小化MSE，那么该预测值应该是所有目标值的均值。但是如果我们试图最小化MAE，那么这个预测就是所有目标值的中位数。我们知道中位数对于离群点比平均值更鲁棒，这使得MAE比MSE更加鲁棒。

使用MAE损失(特别是对于神经网络)的一个大问题是它的梯度始终是相同的，这意味着即使对于小的损失值，其梯度也是大的。这对模型的学习可不好。为了解决这个问题，我们可以使用随着接近最小值而减小的动态学习率。MSE在这种情况下的表现很好，即使采用固定的学习率也会收敛。MSE损失的梯度在损失值较高时会比较大，随着损失接近0时而下降，从而使其在训练结束时更加精确。如果离群点是会影响业务、而且是应该被检测到的异常值，那么我们应该使用MSE。另一方面，如果我们认为离群点仅仅代表数据损坏，那么我们应该选择MAE作为损失。

但是对于这两种模型来说，MAE更加关注的是大部分不离群的情况，而忽略了少部分离群的情况，因为他会尝试去接近中值；但是MSE则会关注离群的情况，因为它被离群点弄糊涂了。所以这两种情况在许多业务中都是不可取的。

| L2损失函数 | L1损失函数 |
| --------- | -------- |
| 不是非常的鲁棒 | 鲁棒 |
| 稳定解 | 不稳定解 |
| 总是一个解 | 可能多个解 |

## 1.4.平滑的平均绝对误差-Huber Loss
Huber Loss对数据离群点的敏感度低于平方误差损失。它在0处也可导。基本上它是绝对误差，当误差很小时，误差是二次形式的。误差何时需要变成二次形式取决于一个超参数$\delta$，该超参数可以进行微调。当$\delta \sim 0$时， Huber Loss接近MAE，当$\delta \sim \infty$（很大的数）时，Huber Loss接近MSE
$$
L_{\delta}(y, f(x))=\begin{cases}
\frac{1}{2}(y-f(x))^2 &\left| y-f(x) \right| \leq \delta \\
\delta \left| y-f(x) \right|-\frac{1}{2}\delta^2 &otherwise
\end{cases}
$$
![images](Images/02/13_01_003.jpg)

使用MAE训练神经网络的一个大问题是经常会遇到很大的梯度，使用梯度下降时可能导致训练结束时错过最小值。对于MSE，梯度会随着损失接近最小值而降低，从而使其更加精确。在这种情况下，Huber Loss可能会非常有用，因为它会使最小值附近弯曲，从而降低梯度。另外它比MSE对异常值更鲁棒。因此，它结合了MSE和MAE的优良特性。但是，Huber Loss的问题是我们可能需要迭代地训练超参数$\delta$。

## 1.5.Log-Cosh Loss
Log-cosh是用于回归任务的另一种损失函数，它比L2更加平滑。Log-cosh是预测误差的双曲余弦的对数
$$L(y,y^p)=\sum_{i=1}^nlog[cosh(y_i^p-y_i)]$$
![images](Images/02/13_01_004.jpg)

$log(cosh(x))$对于小的$x$来说，其大约等于$\frac{x^2}{2}$，而对于大的$x$来说，其大约等于$abs(x) - log(2)$。这意味着'logcosh'的作用大部分与均方误差一样，但不会受到偶尔出现的极端不正确预测的强烈影响。它具有Huber Loss的所有优点，和Huber Loss不同之处在于，其处处二次可导。为什么我们需要二阶导数？许多机器学习模型的实现（如XGBoost）使用牛顿方法来寻找最优解，这就是为什么需要二阶导数（Hessian）的原因。对于像XGBoost这样的机器学习框架，二阶可导函数更有利。

## 1.6.smooth L1 Loss
smooth L1说的是光滑之后的L1
$$
smooth_{L1}(x)=\begin{cases}
0.5x^2 & \left| x \right| < 1 \\
\left| x \right| - 0.5 & otherwise
\end{cases}
$$
![images](Images/02/13_01_005.png)